In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import catalysis
import xarray
import glob
import dask

In [3]:
import os
import dask.distributed

# Edit as desired
threads_per_worker = 1

try:
    c # Already running
except NameError:
    c = dask.distributed.Client(
        n_workers=int(os.environ['PBS_NCPUS'])//threads_per_worker,
        threads_per_worker=threads_per_worker,
        memory_limit=f'{4*threads_per_worker}gb',
        local_directory=os.path.join(os.environ['PBS_JOBFS'],
                                     'dask-worker-space')
    )
c

Client Scheduler: tcp://127.0.0.1:33965 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.00 GB


In [4]:
ds = xarray.open_mfdataset(sorted(glob.glob('/g/data/w35/dh4185/data/AWAP/awra_sm_pct/sm_pct_*.nc')), 
                           combine='nested', 
                           concat_dim='time',
                           chunks={'latitude':20, 'longitude': 100})
ds

<xarray.Dataset>
Dimensions:    (latitude: 681, longitude: 841, time: 39637)
Coordinates:
  * longitude  (longitude) float64 112.0 112.0 112.1 112.2 ... 153.9 153.9 154.0
  * latitude   (latitude) float64 -10.0 -10.05 -10.1 ... -43.9 -43.95 -44.0
  * time       (time) datetime64[ns] 1911-01-01 1911-01-02 ... 2019-07-09
Data variables:
    sm_pct     (time, latitude, longitude) float32 dask.array<chunksize=(365, 20, 100), meta=np.ndarray>
Attributes:
    var_name:       sm_pct
    Conventions:    CF-1.6, ACDD-1.3
    institution:    Bureau of Meteorology
    summary:        Data produced by Bureau of Meteorology Australian Water R...
    url:            http://www.bom.gov.au/water/landscape
    title:          Australian Landscape Water Balance AWRA-L Model Version 6
    email:          awrams@bom.gov.au
    date_created:   2018-11-02T12:18:48
    source:         AWRA-L-v6)
    Description:    This data is provided by the Bureau of Meteorology and ca...
    date_modified:  2018-11-02T12:18:48

In [5]:
catalysis.helpers.dask_report(ds.sm_pct)

Chunk Count: 34335
Chunk Size: 2.93 MB
Graph Size: 68779


In [6]:
r = catalysis.rank.rank_by_dayofyear(ds.sm_pct)
r

<xarray.DataArray 'sm_pct' (time: 39637, latitude: 681, longitude: 841)>
dask.array<group_helper-d3049b2c09f1b4f01ea177f75034caea-<this, shape=(39637, 681, 841), dtype=float32, chunksize=(39637, 20, 100), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 112.0 112.0 112.1 112.2 ... 153.9 153.9 154.0
  * time       (time) datetime64[ns] 1911-01-01 1911-01-02 ... 2019-07-09
  * latitude   (latitude) float64 -10.0 -10.05 -10.1 ... -43.9 -43.95 -44.0
Attributes:
    name:           sm_pct
    units:          fraction of fullness
    long_name:      Total rootzone soil moisture (0-100 cm)
    standard_name:  sm_pct

In [7]:
catalysis.helpers.dask_report(r)

Chunk Count: 315
Chunk Size: 317.10 MB
Graph Size: 70984


In [8]:
r[0,0,0].load()

<xarray.DataArray 'sm_pct' ()>
array(0)
Coordinates:
    longitude  float64 112.0
    time       datetime64[ns] 1911-01-01
    latitude   float64 -10.0
Attributes:
    name:           sm_pct
    units:          fraction of fullness
    long_name:      Total rootzone soil moisture (0-100 cm)
    standard_name:  sm_pct

In [9]:
%%time
#catalysis.io.to_netcdf_chunkwise(r, '~/scratch/test.nc')


CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 5.48 µs


In [10]:
c.cluster.workers

{0: <Nanny: tcp://127.0.0.1:33615, threads: 1>,
 1: <Nanny: tcp://127.0.0.1:34435, threads: 1>,
 2: <Nanny: tcp://127.0.0.1:45825, threads: 1>,
 3: <Nanny: tcp://127.0.0.1:37285, threads: 1>}

In [11]:
def to_netcdf_chunkwise(da, path, complevel=4):
    """
    Save a DataArray to file by calculating each chunk separately (rather than
    submitting the whole Dask graph at once). This may be helpful when chunks
    are large, e.g. doing an operation on dayofyear grouping for a long timeseries.
    """
    ds = xarray.Dataset({da.name: da})

    encoding = {da.name: {
        'zlib': True,
        'shuffle': True,
        'complevel': complevel,
        'chunksizes': da.data.chunksize,
        }}

    f = ds.to_netcdf(path, encoding=encoding, compute=False)

    client = c
    sync = True

    # Run each of the save operations one at a time, then finalize
    old_graph = f.__dask_graph__()
    new_graph = {}
    futures = []
    for k, v in old_graph.items():
        if v[0] == dask.array.core.store_chunk:
            opt, _ = dask.optimization.cull(old_graph, k)
            opt, _ = dask.optimization.fuse(opt, k)
            new_graph[k] = client.get(opt, k, sync=sync)
            futures.append(new_graph[k])
            continue
            
        new_graph[k] = v
    
    # Finalise
    k = list(f.__dask_layers__())
    opt, _ = dask.optimization.cull(new_graph, k)
    opt, _ = dask.optimization.fuse(opt, k)
    ff = client.get(opt, k, sync=sync)
    
    if not sync:
        dask.distributed.wait(ff)


In [ ]:
%%time

to_netcdf_chunkwise(r, '~/scratch/test.nc')
